In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env

m = wt.watertap_setup(dynamic = False)


wt.case_study_trains.train = {"case_study": "santa_barbara",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "santa_barbara", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "seawater"}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.


santa_barbara
------- Adding Unit Processes -------
sw_onshore_intake
ferric_chloride_addition
chlorination
static_mixer
holding_tank
7200.0
media_filtration
anti_scalant_addition
cartridge_filtration
reverse_osmosis
holding_tank_b
3600.0
uv_aop
co2_addition
lime_addition
treated_storage
3600.0
backwash_solids_handling
landfill
municipal_drinking
surface_discharge
-------------------------------------
adding source
----- Connecting Unit Processes -----
seawater ToUnitName --> sw_onshore_intake
sw_onshore_intake ToUnitName --> ferric_chloride_addition
ferric_chloride_addition ToUnitName --> chlorination
chlorination ToUnitName --> static_mixer
static_mixer ToUnitName --> holding_tank
media_filtration ToUnitName --> anti_scalant_addition
media_filtration ToUnitName --> backwash_solids_handling
anti_scalant_addition ToUnitName --> cartridge_filtration
cartridge_filtration ToUnitName --> reverse_osmosis
reverse_osmosis ToUnitName --> holding_tank_b
holding_tank_b ToUnitName --> uv_aop
uv_a

In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

In [4]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("unfixing feed presure and area for", key)
        
wt.run_water_tap(m = m, objective=True)

# prints RO results
print("optimal ro area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print("setting feed presure for", key, "to -->", m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])

wt.run_water_tap(m = m, objective=True, print_model_results="summary", skip_small = True)

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

unfixing feed presure and area for reverse_osmosis
initial inflow to train is relatively small (< 1 m3/s). running model with dummy flows to initialize.
degrees_of_freedom: 2
WaterTAP3 solution optimal
model finished running to initialize conditions. now running with actual inflows.
degrees_of_freedom: 2
WaterTAP3 solution optimal
optimal ro area and pressures:
reverse_osmosis feed pressure 79.33248801931535
reverse_osmosis membrane area 17730.732777715144
setting feed presure for reverse_osmosis to --> 85
degrees_of_freedom: 1
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 2.0085709185785228
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 1.6475042846816659
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 5.206438709376753
----------------------------------------------------------------------
fs.static_mixer
total_cap_investment: 0

In [5]:
# If you need the system recovery to match better.... set a maximum recovery rate.

In [6]:
from pyomo.environ import Constraint
#m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.55) # THIS IS FOR TAMPA BAY
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.50) # THIS IS FOR SANTA BARBARA

In [7]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [8]:
wt.run_water_tap(m = m, objective=True, print_model_results="summary", skip_small = True)
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

degrees_of_freedom: 1
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 2.0085709185785228
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 1.6475042846816659
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 5.206438709376753
----------------------------------------------------------------------
fs.static_mixer
total_cap_investment: 0.1162155841035918
----------------------------------------------------------------------
fs.holding_tank
total_cap_investment: 1.4805906878743877
----------------------------------------------------------------------
fs.media_filtration
total_cap_investment: 2.6868232956463585
----------------------------------------------------------------------
fs.anti_scalant_addition
total_cap_investment: 0.06841919446375062
----------------------------------------------------------------------
fs.cartridge_filtration
t

In [9]:
# RESET AREA TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [10]:
# Readjust recovery constraint and deactivate objective constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery >= 0)
m.fs.objective_function.deactivate()

In [11]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m = m, objective=False, print_model_results="summary", skip_small = True)

degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 2.0085709185785228
----------------------------------------------------------------------
fs.ferric_chloride_addition
total_cap_investment: 1.6475042846816659
----------------------------------------------------------------------
fs.chlorination
total_cap_investment: 5.206438709376753
----------------------------------------------------------------------
fs.static_mixer
total_cap_investment: 0.1162155841035918
----------------------------------------------------------------------
fs.holding_tank
total_cap_investment: 1.4805906878743877
----------------------------------------------------------------------
fs.media_filtration
total_cap_investment: 2.6868232956463585
----------------------------------------------------------------------
fs.anti_scalant_addition
total_cap_investment: 0.06841919446375062
----------------------------------------------------------------------
fs.cartridge_filtration
t

In [12]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m = m, case_study = wt.case_study_trains.source_water["case_study"], 
                                scenario = wt.case_study_trains.source_water["scenario"])

AttributeError: 'SimpleBlock' object has no attribute 'fixed_op_cost_annual'

In [ ]:
# Find bounds for RO area:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).membrane_area[0])
        getattr(m.fs, key).membrane_area.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0] * 1, sense=env.minimize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).membrane_area[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).membrane_area[0]())

        getattr(m.fs, key).membrane_area.fix(stash_value)


In [ ]:
# Find bounds for RO pessure:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).feed.pressure[0])
        getattr(m.fs, key).feed.pressure.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.minimize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).feed.pressure[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).feed.pressure[0]())

        getattr(m.fs, key).feed.pressure.fix(stash_value)

In [ ]:
m.fs.objective_function1.deactivate()

In [ ]:
# run to rest before sensitivity
wt.run_water_tap(m = m, objective=False, skip_small = True)
print("LCOW -->", m.fs.costing.LCOW())

In [ ]:
# sensitivity analyses
sens_df = pd.DataFrame()

lcow_list = []
water_recovery_list = []
scenario_value = []
scenario_name = []
elec_lcow = []

lcow_list.append(value(m.fs.costing.LCOW))
water_recovery_list.append(value(m.fs.costing.system_recovery))
scenario_value.append("n/a")
scenario_name.append("baseline")
elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))

runs_per_scenario = 20

############ onstream_factor 70-100% ############
stash_value = m.fs.costing_param.plant_cap_utilization
scenario = "plant_cap_utilization"
print("-------", scenario, "-------")
ub = 1
lb = 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.costing_param.plant_cap_utilization = i
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_cap_utilization = stash_value    
############################################################    

############ salinity  30-45 ############
stash_value = m.fs.seawater.conc_mass_in[0, "tds"]()
scenario = "inlet salinty 25k-45k mg/L"
print("-------", scenario, "-------")
ub = 25
lb = 45
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.seawater.conc_mass_in[0, "tds"].fix(i) 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.conc_mass_in[0, "tds"].fix(stash_value) 
############################################################

############ inlet flow ############
stash_value = m.fs.seawater.flow_vol_in[0]()
scenario = "inlet flow"
print("-------", scenario, "-------")
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.seawater.flow_vol_in.fix(i) 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.flow_vol_in.fix(stash_value) 
############################################################

############ lifetime years ############

stash_value = value(m.fs.costing_param.plant_lifetime_yrs)
scenario = "lifetime (yrs)"
print("-------", scenario, "-------")
ub = 50
lb = 15
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.costing_param.plant_lifetime_yrs = i 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_lifetime_yrs = stash_value
############################################################

############ elec cost +-20% ############

stash_value = value(m.fs.costing_param.electricity_price)
scenario = "electricity price +- 20%"
print("-------", scenario, "-------")
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.costing_param.electricity_price = i 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.electricity_price = stash_value
############################################################

############ RO scenarios --> pressure, membrane area, replacement rate% ############

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        area = value(getattr(m.fs, key).membrane_area[0])

        scenario_dict = {"membrane_area" : [-area*0.2, area*0.2], 
                         "pressure": [-15, 10], 
                         "factor_membrane_replacement": [-0.1, 0.1]}
        
        for scenario in scenario_dict.keys():
            if scenario == "pressure":
                stash_value = value(getattr(getattr(getattr(m.fs, key), "feed"), scenario)[0])
            else:
                stash_value = value(getattr(getattr(m.fs, key), scenario)[0])

            print("-------", scenario, "-------")
            ub = stash_value + scenario_dict[scenario][1]
            lb = stash_value + scenario_dict[scenario][0]
            step = (ub - lb) / runs_per_scenario
            
            for i in np.arange(lb, ub + step, step):
                if scenario == "pressure":
                    getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(i)
                else:
                    getattr(getattr(m.fs, key), scenario).fix(i)
                
                wt.run_water_tap(m = m, objective=False, skip_small = True)
                print("LCOW -->", m.fs.costing.LCOW())
    
                lcow_list.append(value(m.fs.costing.LCOW))
                water_recovery_list.append(value(m.fs.costing.system_recovery))
                scenario_value.append(i)
                scenario_name.append(key + "_" + scenario)
                elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
            if scenario == "pressure":
                getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(stash_value)
            else:
                getattr(getattr(m.fs, key), scenario).fix(stash_value)
                            
############################################################

# final run to get baseline numbers again
wt.run_water_tap(m = m, objective=False, skip_small = True)

sens_df["lcow"] = lcow_list
sens_df["water_recovery"] =  water_recovery_list
sens_df["elec_lcow"] =  elec_lcow
sens_df["scenario_value"] = scenario_value
sens_df["scenario_name"] = scenario_name
sens_df["lcow_difference"] =  sens_df.lcow - value(m.fs.costing.LCOW)
sens_df["water_recovery_difference"] = (sens_df.water_recovery - value(m.fs.costing.system_recovery))
sens_df["elec_lcow_difference"] = (sens_df.elec_lcow - value(m.fs.costing.elec_frac_LCOW))

sens_df.elec_lcow = sens_df.elec_lcow * 100
sens_df.water_recovery = sens_df.water_recovery * 100


In [ ]:
sens_df.to_csv("results/case_studies/santa_barbara_baseline_sensitivity.csv")

In [ ]:
############ inlet flow ############
stash_value = m.fs.seawater.flow_vol_in[0]()
scenario = "inlet flow"
print("-------", scenario, "-------")
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub, step):
    m.fs.seawater.flow_vol_in.fix(i) 
    wt.run_water_tap(m = m, objective=False)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.flow_vol_in.fix(stash_value) 
############################################################

In [ ]:
ub

In [ ]:
lb

In [ ]:
i

In [ ]:
m.fs.seawater.flow_vol_in[0]()

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)